# Phase2 Scattering Router A/B (Colab ready)
- Clone repo (if needed)
- Install deps
- Train baseline router
- Train scattering router (`--use-scattering-router`)
- Show stdout/stderr if失敗
- Summarize loss/PPL from logs


In [ ]:
import subprocess, pathlib
repo_url = "https://github.com/neko-jpg/Project-ResNet-BK-An-O-N-Language-Model-Architecture.git"
cwd = pathlib.Path('.').resolve()
if (cwd / '.git').exists():
    repo_root = cwd
else:
    repo_root = cwd / 'Project-ResNet-BK-An-O-N-Language-Model-Architecture'
    if not repo_root.exists():
        print(f'Cloning {repo_url} into {repo_root} ...')
        subprocess.run(['git', 'clone', repo_url, str(repo_root)], check=True)
    else:
        print(f'Using existing clone at {repo_root}')
print('Repo root:', repo_root)


In [ ]:
import subprocess, sys
print('Installing requirements...')
res = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], cwd=repo_root, text=True, capture_output=True)
print('returncode', res.returncode)
print(res.stdout)
print(res.stderr)
assert res.returncode == 0, 'pip install failed'


In [ ]:
import subprocess, sys
baseline_dir = repo_root / 'checkpoints' / 'phase2_baseline'
baseline_dir.mkdir(parents=True, exist_ok=True)
cmd = [sys.executable, 'train.py', '--config-preset', 'baseline', '--save-dir', str(baseline_dir)]
print('Running baseline router:', ' '.join(cmd))
res = subprocess.run(cmd, cwd=repo_root, text=True, capture_output=True)
print('returncode', res.returncode)
print('--- stdout ---')
print(res.stdout)
print('--- stderr ---')
print(res.stderr)
assert res.returncode == 0, 'baseline training failed'
print('Logs in', baseline_dir / 'logs')


In [ ]:
import subprocess, sys
scatt_dir = repo_root / 'checkpoints' / 'phase2_scattering'
scatt_dir.mkdir(parents=True, exist_ok=True)
cmd = [sys.executable, 'train.py', '--config-preset', 'baseline', '--use-scattering-router', '--scattering-scale', '0.1', '--save-dir', str(scatt_dir)]
print('Running scattering router:', ' '.join(cmd))
res = subprocess.run(cmd, cwd=repo_root, text=True, capture_output=True)
print('returncode', res.returncode)
print('--- stdout ---')
print(res.stdout)
print('--- stderr ---')
print(res.stderr)
assert res.returncode == 0, 'scattering training failed'
print('Logs in', scatt_dir / 'logs')


In [ ]:
import csv, pathlib

def latest_csv(log_dir: pathlib.Path):
    files = sorted(log_dir.glob('*.csv'), key=lambda p: p.stat().st_mtime, reverse=True)
    return files[0] if files else None

def read_last_row(csv_path: pathlib.Path):
    with csv_path.open() as f:
        rows = list(csv.DictReader(f))
    return rows[-1] if rows else None

def summarize(run_name, save_dir: pathlib.Path):
    log_dir = save_dir / 'logs'
    csv_path = latest_csv(log_dir)
    if not csv_path:
        print(f"[{run_name}] no CSV in {log_dir}")
        return None
    last = read_last_row(csv_path)
    if not last:
        print(f"[{run_name}] empty CSV {csv_path}")
        return None
    out = {
        'path': str(csv_path),
        'step': int(last['step']),
        'epoch': int(last['epoch']),
        'loss': float(last['loss']),
        'ppl': float(last['perplexity']),
        'grad_norm': float(last['grad_norm']),
        'nan': int(last.get('num_nan_grads', 0) or 0),
        'inf': int(last.get('num_inf_grads', 0) or 0),
    }
    print(f"[{run_name}] step {out['step']} epoch {out['epoch']} loss {out['loss']:.4f} ppl {out['ppl']:.1f} grad_norm {out['grad_norm']:.3f} nan {out['nan']} inf {out['inf']} ({csv_path.name})")
    return out

base_summary = summarize('baseline', baseline_dir)
scatt_summary = summarize('scattering', scatt_dir)

if base_summary and scatt_summary:
    print(f"Δloss (scatt-base): {scatt_summary['loss'] - base_summary['loss']:+.4f}, Δppl: {scatt_summary['ppl'] - base_summary['ppl']:+.1f}")
else:
    print('Could not compare; missing summaries.')
